<a href="https://colab.research.google.com/github/flying-smoothly/AI_vision/blob/master/gpt_apipng%2Bcsv%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip uninstall openai

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [3]:
!pip install python-dotenv

In [4]:
pip install opencv-python
pip install opencv-python-headless

SyntaxError: invalid syntax (<ipython-input-4-75d6990a88f1>, line 1)

In [5]:
import os
import cv2
import openai
from openai import OpenAI
import requests
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import io
from io import BytesIO
import base64
from dotenv import load_dotenv, find_dotenv

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
load_dotenv('/content/drive/MyDrive/key.env')

openai.api_key = os.getenv('OPENAI_API_KEY') #os 모듈을 사용하여 환경 변수에서 API 키를 불러옵니다.

if openai.api_key:
    print("API 키가 성공적으로 설정되었습니다.")
else:
    print("API 키를 설정하는 데 실패했습니다.")

client=OpenAI()

API 키가 성공적으로 설정되었습니다.


In [9]:
# Function to encode a file
def encode_file(file_path):
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Paths to your files
image_path = "/content/drive/MyDrive/depth.bag/depth_Depth_31135.png"
csv1_path = "/content/drive/MyDrive/depth.bag/depth_Depth_31135_metadata.csv"
csv2_path = "/content/drive/MyDrive/depth.bag/spray_depth_metrics.csv"

# Encoding the files
base64_image = encode_file(image_path)
base64_csv1 = encode_file(csv1_path)
base64_csv2 = encode_file(csv2_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "PNG 파일에 depth camera로 찍은 이미지의 깊이 정보와 RGB 정보가 저장되어 있어. 이미지는 스프레이를 찍은 거고 나는 스프레이의 깊이 정보를 알고 싶어. 스프레이가 카메라로부터 얼마나 떨어져 있었는지 png 파일을 읽어서 말해줄 수 있어? 얼마나 떨어져 있는지만 말해주면 돼."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                },
                {
                    "type": "file",
                    "file": {
                        "filename": "depth_Depth_31135_metadata.csv",
                        "filetype": "csv",
                        "content": base64_csv1
                    }
                },
                {
                    "type": "file",
                    "file": {
                        "filename": "spray_depth_metrics.csv",
                        "filetype": "csv",
                        "content": base64_csv2
                    }
                }
            ]
        }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'error': {'message': 'Invalid chat format. Content blocks are expected to be either text or image_url type.', 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [14]:
import json

# Function to encode a file
def encode_file(file_path):
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Paths to your files
image_path = "/content/drive/MyDrive/depth.bag/depth_Depth_31135.png"
csv1_path = "/content/drive/MyDrive/depth.bag/depth_Depth_31135_metadata.csv"
csv2_path = "/content/drive/MyDrive/depth.bag/spray_depth_metrics.csv"

# Encoding the files
base64_image = encode_file(image_path)
base64_csv1 = encode_file(csv1_path)
base64_csv2 = encode_file(csv2_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """"Can you tell me the location of the cat on the image, very accurately, ensuring that the area covers the entire object (cat).
                    Share the x_min, y_min, x_max, y_max in 0-1 normalized space. Only return the numbers, nothing else. Calculate the pixel values from the normalized values. The total width is 424 and the total height is 240.
                    Then, using the calculated pixel coordinates, detect the object within the image with a bounding box." png,csv 파일에 depth camera로 찍은 이미지의 깊이 정보와 RGB 정보가 저장되어 있어. 이미지는 스프레이를 찍은 거고 나는 스프레이의 깊이 정보를 알고 싶어.
                    스프레이가 카메라로부터 얼마나 떨어져 있었는지 png, csv 파일을 읽어서 말해줄 수 있어? 얼마나 떨어져 있는지만 말해주면 돼."""
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                },
                {
                    "type": "text",
                    "text": f"첫 번째 CSV 파일 (depth_Depth_31135_metadata.csv) 내용:\n{base64_csv1}"
                },
                {
                    "type": "text",
                    "text": f"두 번째 CSV 파일 (spray_depth_metrics.csv) 내용:\n{base64_csv2}"
                }
            ]
        }
    ],
    "max_tokens": 400
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# Parse the JSON response
response_json = response.json()

# Extract the 'content' value
content = response_json['choices'][0]['message']['content']

# Print the 'content' value
print(content)

# If the response is incomplete, continue the conversation to get more information
while response_json['choices'][0]['finish_reason'] == 'length':
    payload['messages'].append({"role": "assistant", "content": content})
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    response_json = response.json()
    content = response_json['choices'][0]['message']['content']
    print(content)


제시하신 데이터를 바탕으로 깊이 정보를 분석해 보았습니다. 그러나 텍스트 형태의 CSV 파일을 통해서는 깊이 이미지 내 특정 객체(예: 스프레이)의 위치 정보를 직접적으로 식별하고 계산하기 어려울 수 있습니다. 

일반적으로 RGB와 깊이 정보를 저장하고 있는 이미지를 나타낸 후, 깊이 값(단위: mm 또는 m)을 통해 특정 객체의 거리를 찾아내려면 이미지 분석 및 객체 식별 과정을 거쳐야 합니다. 이 과정에서는 특정 객체를 인식한 후, 해당 객체의 평균 깊이를 계산하게 됩니다.

제공된 이미지를 분석해본 결과, 빨간색에서 파란색으로의 그라데이션이 깊이 정보를 나타내는 것 같습니다. 검정색은 데이터가 없거나 노이즈로 인해 깊이 정보를 얻을 수 없는 부분일 수 있습니다. 

스프레이의 정확한 포지션을 정의하고 나서, 해당 부분의 평균 깊이 정보를 이용해 카메라로부터의 거리를 산출해야 합니다.

**CSV 파일 예시:**
```csv
디바이스 정보:
유형: 이어링모델X
HW ID: 0B07
시리얼 넘버: 1366220739800
펌웨어 버전: 5.16.0.1
스트리밍 프로파일:
스트림 | 포맷 | 해상도 | FPS
깊이 | Z16 | 848x480 | 30
인프라레드 1 | Y8 | 848x480 | 30

환경:
평면 적합 거리(mm), N/A
지면-진실 거리(mm), N/A

샘플 ID, 프레임 #, 타임스탬프 (ms), 필 레이트 %, z 정확도 %, 평면 적합 RMS 오류 %, 서브픽셀 RMS 오류
이미지를 통해 스프레이의 깊이 정보를 분석하려면, 깊이 이미지의 특정 좌표에서의 값을 추출해야 합니다. 그러나 현재 주어진 데이터는 전체 이미지를 기반으로 한 것으로, 특정 객체에 대해 이야기를 하려면 상세한 이미지 좌표나 추가 정보를 통해 객체 위치를 도출해야 합니다. 즉, 아래의 과정이 필요할 것입니다.

1. **이미지 분석을 통한 객체 위치 추출**:
   - 스프레이 위치를 특정할 수 있다면, 해당 좌표의 깊이 값을 직접 확인할 수 있습니다.
   
2

KeyboardInterrupt: 

In [ ]:
#####################################################
##               Read bag from file                ##
#####################################################


# First import library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
# Import argparse for command-line options
import argparse
# Import os.path for file path manipulation
import os.path

# Create object for parsing command-line options
parser = argparse.ArgumentParser(description="Read recorded bag file and display depth stream in jet colormap.\
                                Remember to change the stream fps and format to match the recorded.")
# Add argument which takes path to a bag file as an input
parser.add_argument("-i", "--input", type=str, help="Path to the bag file")
# Parse the command line arguments to an object
args = parser.parse_args()
# Safety if no parameter have been given
if not args.input:
    print("No input paramater have been given.")
    print("For help type --help")
    exit()
# Check if the given file have bag extension
if os.path.splitext(args.input)[1] != ".bag":
    print("The given file is not of correct file format.")
    print("Only .bag files are accepted")
    exit()
try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, args.input)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)

    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
        # Get frameset of depth
        frames = pipeline.wait_for_frames()

        # Get depth frame
        depth_frame = frames.get_depth_frame()

        # Colorize depth frame to jet colormap
        depth_color_frame = colorizer.colorize(depth_frame)

        # Convert depth_frame to numpy array to render image in opencv
        depth_color_image = np.asanyarray(depth_color_frame.get_data())

        # Render image in opencv window
        cv2.imshow("Depth Stream", depth_color_image)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# 이미지 읽기
image_path = '/content/drive/MyDrive/LLM/cat6.jpeg'
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인
if image is None:
    print("Error: Unable to load image. Please check the file path.")
else:
    # 예시로 설정한 컵과 손의 바운딩 박스 좌표 (직접 정의 필요)
    cat_box = (96, 90, 247, 426)  # 고양이 바운딩 박스 좌표
    #hand_box = (830, 700, 960, 820)  # 손 바운딩 박스 좌표

    # 바운딩 박스 그리기
    cv2.rectangle(image, (cat_box[0], cat_box[1]), (cat_box[2], cat_box[3]), (0, 255, 0), 2)
    #cv2.rectangle(image, (hand_box[0], hand_box[1]), (hand_box[2], hand_box[3]), (255, 0, 0), 2)

    # 결과 이미지 보여주기
    cv2_imshow(image)

In [ ]:
# 이미지를 읽어옵니다
image_path = '/content/drive/MyDrive/LLM/cat1.jpg'
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인합니다
if image is None:
    raise ValueError(f"이미지를 로드할 수 없습니다: {image_path}")

# 이미지를 그레이스케일로 변환합니다
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 노이즈 제거를 위해 Gaussian 블러링을 적용합니다
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# 이진화하여 컵을 추출합니다
_, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 모폴로지 연산을 사용하여 객체를 확장합니다
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
morphed = cv2.dilate(thresh, kernel, iterations=2)

# 외곽선을 찾습니다
contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 외곽선을 감싸는 직사각형을 계산하고 이미지에 그립니다
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    # 필터링 조건: 특정 크기 이상의 컨투어만 처리합니다
    if w > 50 and h > 50:
        # 바운딩 박스를 약간 확장합니다
        padding = 10
        x, y, w, h = x - padding, y - padding, w + 2 * padding, h + 2 * padding

        # 정규화된 좌표 출력
        print(f"Cup {i+1}:")
        print(f"Top-left: ({x}, {y})")
        print(f"Top-right: ({x+w}, {y})")
        print(f"Bottom-left: ({x}, {y + h})")
        print(f"Bottom-right: ({x + w}, {y + h})")
        print()

        # 원본 이미지에 사각형 그리기
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 이미지를 출력합니다
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축을 끕니다
plt.show()